In [1]:
import time
from collections import namedtuple

import tensorflow as tf
import numpy as np

In [2]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1240_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1240_l512.ckpt"

In [3]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [4]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки"):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [6]:
batch_size = 100        # Размер пакета
num_steps = 100         # Шагов в пакете
lstm_size = 512         # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [8]:
with open('Data/stephen_king_it.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [10]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # Мы будем использовать эту же сеть для сэмплирования (генерации текста),
        # при этом будем подавать по одному символу за один раз
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Строим LSTM ячейку
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Прогоняем данные через RNN и собираем результаты
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [14]:
def build_inputs(batch_size, num_steps):
    ''' Определяем placeholder'ы для входных, целевых данных, а также вероятности drop out
    
        Аргументы
        ---------
        batch_size: Batch size, количество последовательностей в пакете
        num_steps: Sequence length, сколько "шагов" делаем в пакете
        
    '''
    # Объявляем placeholder'ы
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Placeholder для вероятности drop out
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [16]:
def build_output(lstm_output, in_size, out_size):
    ''' Строим softmax слой и возвращаем результат его работы.
    
        Аргументы
        ---------
        
        x: Входящий от LSTM тензор
        in_size: Размер входящего тензора, (кол-во LSTM юнитов скрытого слоя)
        out_size: Размер softmax слоя (объем словаря)
    
    '''

    # вытягиваем и решэйпим тензор, выполняя преобразование 3D -> 2D
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Соединяем результат LTSM слоев с softmax слоем
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Считаем logit-функцию
    logits = tf.matmul(x, softmax_w) + softmax_b
    # Используем функцию softmax для получения предсказания
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [18]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Считаем функцию потери на основании значений logit-функции и целевых значений.
    
        Аргументы
        ---------
        logits: значение logit-функции
        targets: целевые значения, с которыми сравниваем предсказания
        lstm_size: Количество юнитов в LSTM слое
        num_classes: Количество классов в целевых значениях (размер словаря)
        
    '''
    # Делаем one-hot кодирование целевых значений и решейпим по образу и подобию logits
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Считаем значение функции потери softmax cross entropy loss и возвращаем среднее значение
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [20]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Строим оптимизатор для обучения, используя обрезку градиента.
    
        Arguments:
        loss: значение функции потери
        learning_rate: параметр скорости обучения
    
    '''
    
    # Оптимизатор для обучения, обрезка градиента для контроля "взрывающихся" градиентов
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [21]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки пелелали на ветон селани стовосна кале нена провиниси санос сто вита нел сти като наме то нистенонась нос пело врилили пракал сто стов се палоло, ка налил сос та правана, и порадети, но проди подаль о стум слостони, к оридола, перутана валонь водолол вора слати стето нен весеть ва те прасталос стовани ка подерена, но дета, кестовали нала ном намо нали не педальлис полотьло пета намила.


 Нель поди стули насни не стот рали в ратот санотали, что с палелань ве теро продо сала сел насте перина волито ну вес подалил и кедаластола ва пиринала сто но са пасет ролана ста тато пел сено порулоста. Писто прано на прелали снелом вастини преснать илена. Ол недить сасли ва стуто тенос, она прулона, чеконати ватато стол ни но не носа тель, накони поделасиси ну вас вать пила вас на неле ногат пета наль, потити, сототь ото сновале, чостом ветон сот неми сесл

In [22]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки, но они сталала ворит не начил по не могу, наких казалось с поровом, накакал он в полому не в кресто в потене столони, что она себяла не постеринали сторен на нем поралать в текого поспедноги. Но положения не в куго перучитосто. В кототой с не порнали не вельчеть собя поделеть стурать, что в когда постратия в подум в домей, который семи от не порнила. Они том но водена староть восель стерилась следала пристать но в премому и подерели выдали в трума. В семом подамули, к ну восту ина ни все стревения в так не теля с него он совирол себя ни отручал на ведере вотом, на така в подвитал на в домую чаль оне сольцо на сколок служной кат стонь, который призивил они не в пакоми, которые вскака и верная воскатил не провить в как славетерь вередатальния подеть не столание сомотоние слушие.

– То мене содер всемну непорнел он сливой вы стукала, и в тат по

In [23]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки.

Откид игрулся под канимором, продрежился привелини с положащно с седоваде станому. Но в тем друг он удился в кане вышет с канаме из прадолка. Потым возворот под водом прошле, не выразнылась со дрегой словном всорнать.

– То смогра тель на стала, подалаясь, потому что не все постоила его в колоденя.

– Поставил он и пологив и сказала тебя на него но стала не следую, как обез вольстая.

Потре передали его, как вернул сторону потрогом придавленных кровной, не покалился подумания, на тем, что они оставлесь в сколеко по пальной вольдой и поспили на том прослоше. Постоят он вернул со сторону она, когда он смогрил серебу соврись и неточку с пододавителество перекоритке, и не меня но отваловало стороны в траности и откратившесь с навости с ним, как водол стола с ободной времени, и продились на стаканут и плека на колеста, и вскотик в коросе, прошо

In [24]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab))
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1240_l512.ckpt
Но когда дело дошло до заказов, долговязый мужчина попросил стакан минералки, но смотрел в темным. Стрики, ингалера в каталь и парно из мосте, которые не мог с сломить стеле..

– Я дажа вы могу и подумать?

– Давие то, что его могу не видить выследить начастника с парки. Прошен в столоне вызвинит, к нем, когда наборется отчастно в семней прошин.

– Нет.

– Ду. – Прошил во после тему, что, что он усили. Или началось в петеле от этом. Вы полома по вередение и од полосит собраться, и потом чем в этом не малоечный поробланки слова петел собретнее ни следовитель.

Он совсем водром сторон в глаза.

– Нет, не забывий. Я должен подель в том, что он в полей, почем на потом стоят, послу он посто в тель с нах. И ты направится в тебя и в этот, которая страха ни которых ночь с нем, но но полого в пасни подняли на мества.

– Ты в этом мальчика, когда на метя, когда так возможита, – на смоюти. Он умее смерило его случие. Потом прос